# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 9 2023 11:33AM,259017,10,MSP219348,"Methapharm, Inc.",Released
1,Mar 9 2023 11:33AM,259017,10,MSP219350,"Methapharm, Inc.",Released
2,Mar 9 2023 11:33AM,259017,10,MSP219351,"Methapharm, Inc.",Released
3,Mar 9 2023 11:33AM,259017,10,MSP219352,"Methapharm, Inc.",Released
4,Mar 9 2023 11:33AM,259017,10,MSP219354,"Methapharm, Inc.",Released
5,Mar 9 2023 11:33AM,259017,10,MSP219339,"Methapharm, Inc.",Released
6,Mar 9 2023 11:33AM,259017,10,MSP219341,"Methapharm, Inc.",Released
7,Mar 9 2023 11:33AM,259017,10,MSP219342,"Methapharm, Inc.",Released
8,Mar 9 2023 11:33AM,259017,10,MSP219343,"Methapharm, Inc.",Released
9,Mar 9 2023 11:33AM,259017,10,MSP219346,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,259013,Released,1
23,259014,Released,1
24,259015,Executing,1
25,259016,Released,4
26,259017,Released,13


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259013,NaN,NaN,1.0
259014,NaN,NaN,1.0
259015,NaN,1.0,NaN
259016,NaN,NaN,4.0
259017,NaN,NaN,13.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258866,0.0,0.0,17.0
258960,0.0,0.0,1.0
258964,0.0,0.0,7.0
258975,0.0,0.0,1.0
258979,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258866,0,0,17
258960,0,0,1
258964,0,0,7
258975,0,0,1
258979,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258866,0,0,17
1,258960,0,0,1
2,258964,0,0,7
3,258975,0,0,1
4,258979,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258866,,,17
1,258960,,,1
2,258964,,,7
3,258975,,,1
4,258979,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 9 2023 11:33AM,259017,10,"Methapharm, Inc."
13,Mar 9 2023 11:27AM,259016,10,"Methapharm, Inc."
17,Mar 9 2023 11:04AM,259015,21,"NBTY Global, Inc."
18,Mar 9 2023 11:04AM,259014,19,"NAPP Technologies, LLC"
19,Mar 9 2023 10:57AM,259013,10,"Senseonics, Incorporated"
20,Mar 9 2023 10:47AM,259011,10,"Senseonics, Incorporated"
21,Mar 9 2023 10:02AM,259008,21,"NBTY Global, Inc."
22,Mar 9 2023 9:55AM,259005,10,Eywa Pharma Inc.
35,Mar 9 2023 9:47AM,259002,15,"Person & Covey, Inc."
41,Mar 9 2023 9:37AM,258995,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 9 2023 11:33AM,259017,10,"Methapharm, Inc.",,,13
1,Mar 9 2023 11:27AM,259016,10,"Methapharm, Inc.",,,4
2,Mar 9 2023 11:04AM,259015,21,"NBTY Global, Inc.",,1,
3,Mar 9 2023 11:04AM,259014,19,"NAPP Technologies, LLC",,,1
4,Mar 9 2023 10:57AM,259013,10,"Senseonics, Incorporated",,,1
5,Mar 9 2023 10:47AM,259011,10,"Senseonics, Incorporated",,,1
6,Mar 9 2023 10:02AM,259008,21,"NBTY Global, Inc.",,1,
7,Mar 9 2023 9:55AM,259005,10,Eywa Pharma Inc.,,2,11
8,Mar 9 2023 9:47AM,259002,15,"Person & Covey, Inc.",,,6
9,Mar 9 2023 9:37AM,258995,10,Bio-PRF,,5,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 9 2023 11:33AM,259017,10,"Methapharm, Inc.",13,,
1,Mar 9 2023 11:27AM,259016,10,"Methapharm, Inc.",4,,
2,Mar 9 2023 11:04AM,259015,21,"NBTY Global, Inc.",,1,
3,Mar 9 2023 11:04AM,259014,19,"NAPP Technologies, LLC",1,,
4,Mar 9 2023 10:57AM,259013,10,"Senseonics, Incorporated",1,,
5,Mar 9 2023 10:47AM,259011,10,"Senseonics, Incorporated",1,,
6,Mar 9 2023 10:02AM,259008,21,"NBTY Global, Inc.",,1,
7,Mar 9 2023 9:55AM,259005,10,Eywa Pharma Inc.,11,2,
8,Mar 9 2023 9:47AM,259002,15,"Person & Covey, Inc.",6,,
9,Mar 9 2023 9:37AM,258995,10,Bio-PRF,10,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 9 2023 11:33AM,259017,10,"Methapharm, Inc.",13,,
1,Mar 9 2023 11:27AM,259016,10,"Methapharm, Inc.",4,,
2,Mar 9 2023 11:04AM,259015,21,"NBTY Global, Inc.",,1,
3,Mar 9 2023 11:04AM,259014,19,"NAPP Technologies, LLC",1,,
4,Mar 9 2023 10:57AM,259013,10,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 9 2023 11:33AM,259017,10,"Methapharm, Inc.",13.0,NaN,NaN
1,Mar 9 2023 11:27AM,259016,10,"Methapharm, Inc.",4.0,NaN,NaN
2,Mar 9 2023 11:04AM,259015,21,"NBTY Global, Inc.",NaN,1.0,NaN
3,Mar 9 2023 11:04AM,259014,19,"NAPP Technologies, LLC",1.0,NaN,NaN
4,Mar 9 2023 10:57AM,259013,10,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 9 2023 11:33AM,259017,10,"Methapharm, Inc.",13.0,0.0,0.0
1,Mar 9 2023 11:27AM,259016,10,"Methapharm, Inc.",4.0,0.0,0.0
2,Mar 9 2023 11:04AM,259015,21,"NBTY Global, Inc.",0.0,1.0,0.0
3,Mar 9 2023 11:04AM,259014,19,"NAPP Technologies, LLC",1.0,0.0,0.0
4,Mar 9 2023 10:57AM,259013,10,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3107935,61.0,7.0,0.0
15,517868,23.0,0.0,0.0
19,1294912,40.0,38.0,15.0
21,518023,0.0,2.0,0.0
22,258993,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3107935,61.0,7.0,0.0
1,15,517868,23.0,0.0,0.0
2,19,1294912,40.0,38.0,15.0
3,21,518023,0.0,2.0,0.0
4,22,258993,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,61.0,7.0,0.0
1,15,23.0,0.0,0.0
2,19,40.0,38.0,15.0
3,21,0.0,2.0,0.0
4,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,61.0
1,15,Released,23.0
2,19,Released,40.0
3,21,Released,0.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,21,22
Status,,,,,
Completed,0.0,0.0,15.0,0.0,0.0
Executing,7.0,0.0,38.0,2.0,0.0
Released,61.0,23.0,40.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,21,22
0,Completed,0.0,0.0,15.0,0.0,0.0
1,Executing,7.0,0.0,38.0,2.0,0.0
2,Released,61.0,23.0,40.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,21,22
0,Completed,0.0,0.0,15.0,0.0,0.0
1,Executing,7.0,0.0,38.0,2.0,0.0
2,Released,61.0,23.0,40.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()